### Project 1 - 불량을 야기하는 변수를 찾고 불량품을 낮추기 위한 분석 진행
- 타겟 변수: 불량여부를 나타내는 이진데이터
- 설명 변수: 변수에 대한 설명이 없는 비식별화된 20개의 변수

In [2]:
# 필요한 라이브러리 불러오기 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.metrics import classification_report, confusion_matrix, roc_curve 
from imblearn.under_sampling import *
from imblearn.over_sampling import *


In [4]:
# 데이터 불러오기 
defect = pd.read_csv("./defect.csv")
defect.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,Y
0,0.202296,0.003560,0.411261,0.015348,0.693147,0.036360,0.343512,0.028087,0.681974,0.295769,...,0.682731,0.249262,0.681974,0.016330,0.242926,0.557461,0.028087,0.682731,0.036360,0
1,0.006836,0.007688,0.597304,0.015348,0.686838,0.067407,0.347869,0.028640,0.497403,0.297943,...,0.680891,0.249262,0.497403,0.670040,0.245876,0.561336,0.028640,0.680891,0.067407,0
2,0.609621,0.001461,0.466186,0.015348,0.693090,0.018944,0.345032,0.024502,0.647685,0.298255,...,0.685525,0.249262,0.647685,0.648486,0.243602,0.559431,0.024502,0.685525,0.018944,0
3,0.074007,0.003072,0.535876,0.015348,0.693147,0.031475,0.347502,0.025393,0.587787,0.296057,...,0.677980,0.249262,0.575364,0.632563,0.246078,0.556761,0.025393,0.677980,0.031475,0
4,0.198356,0.007020,0.416238,0.015348,0.686838,0.061888,0.347441,0.028450,0.537143,0.298167,...,0.673286,0.249262,0.537143,0.671124,0.245505,0.561695,0.028450,0.673286,0.061888,0


In [5]:
defect.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X12,X13,X14,X15,X16,X17,X18,X19,X20,Y
count,527000.000000,527000.000000,527000.000000,5.270000e+05,527000.000000,527000.000000,527000.000000,527000.000000,527000.000000,527000.000000,...,527000.000000,5.270000e+05,527000.000000,527000.000000,527000.000000,527000.000000,527000.000000,527000.000000,527000.000000,527000.000000
mean,0.375129,0.003963,0.455679,1.534800e-02,0.638446,0.037449,0.348374,0.025152,0.602939,0.299989,...,0.680077,2.492619e-01,0.601662,0.603826,0.247010,0.559367,0.025152,0.680077,0.037449,0.108159
std,0.200043,0.004316,0.105150,2.081670e-17,0.139334,0.016994,0.028971,0.003820,0.071272,0.033502,...,0.005244,1.387780e-16,0.065753,0.165551,0.030353,0.013077,0.003820,0.005244,0.016994,0.310582
min,0.000000,0.000000,0.000000,1.534800e-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.492619e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.204692,0.002366,0.415990,1.534800e-02,0.688810,0.026023,0.344457,0.024164,0.562785,0.295929,...,0.676136,2.492619e-01,0.562785,0.623615,0.243377,0.556788,0.024164,0.676136,0.026023,0.000000
50%,0.399744,0.003523,0.435739,1.534800e-02,0.693147,0.034280,0.345721,0.025805,0.624154,0.296921,...,0.680586,2.492619e-01,0.612178,0.645785,0.244290,0.557708,0.025805,0.680586,0.034280,0.000000
75%,0.545196,0.005094,0.530429,1.534800e-02,0.693147,0.048085,0.346949,0.028069,0.659246,0.297723,...,0.684354,2.492619e-01,0.647685,0.668555,0.245254,0.559589,0.028069,0.684354,0.048085,0.000000
max,0.693147,0.693147,0.693147,1.534800e-02,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,...,0.693147,2.492619e-01,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,0.693147,1.000000


In [6]:
defect.columns

Index(['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11',
       'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 'X20', 'Y'],
      dtype='object')

In [7]:
# null 값 확인
defect.isnull().sum().max()

0

In [8]:
print('Defect',round(defect['Y'].value_counts()[1]/len(defect) * 100,2), '% of the dataset')

Defect 10.82 % of the dataset


In [9]:
plt.plot(df)

KeyboardInterrupt: 

: 

: 